In [234]:
import pandas
import numpy as np
import sklearn.metrics

In [5]:
train_data = pandas.read_csv('train.csv', sep=',')
train_data

,user_id,course_id
0,39972,34
1,56815,51
2,63734,20
3,17896,81
4,36961,64
...,...,...
10267,10520,10
10268,62231,40
10269,150361,148
10270,4182,7


In [208]:
best_couses = list(train_data.groupby('course_id').user_id.nunique().sort_values(ascending=False).head(3).keys())
#best_couses = [4, 8, 15]
best_couses

[7, 1, 15]

In [9]:
train_data['num'] = 1
train_data

,user_id,course_id,num
0,39972,34,1
1,56815,51,1
2,63734,20,1
3,17896,81,1
4,36961,64,1
...,...,...,...
10267,10520,10,1
10268,62231,40,1
10269,150361,148,1
10270,4182,7,1


In [209]:
matrix = train_data.pivot(index="course_id", columns="user_id", values='num').fillna(0)
matrix

user_id,4,78,81,83,84,120,150,170,171,188,...,186419,186420,186469,186547,186569,186582,186668,186835,187554,187639
course_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0


In [237]:
# Singular value decomposition
u, s, vh = np.linalg.svd(matrix.values, full_matrices=False)

In [236]:
def cosine_similarity_orig(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

def cosine_similarity(v,u):
    return sklearn.metrics.pairwise.euclidean_distances(v.reshape(1, -1), u.reshape(1, -1))
 
highest_similarity = -np.inf
highest_sim_col = -1
for col in range(0, vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
 
print("Column %d is most similar to column 0" % highest_sim_col)

Column 2786 is most similar to column 0


In [232]:
#Normalizing the matrix(subtract mean off)
normalised_mat = matrix.values - np.asarray([(np.mean(matrix.values, 1))]).T

#Computing the Singular Value Decomposition (SVD)
#A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
u, s, vh = np.linalg.svd(normalised_mat, full_matrices=False)

In [228]:
vh.shape

(171, 4862)

In [238]:
def is_new_user(user_id):
    return train_data.loc[train_data['user_id'] == user_id]['user_id'].count() == 0

In [239]:
def get_user_col_by_id(user_id):
    return matrix.columns.get_loc(user_id)

In [240]:
def get_sims_by_user_id(user_id):
    user_col_num = get_user_col_by_id(user_id)
    sims = {}
    for col in range(1,vh.shape[1]):
        similarity = cosine_similarity(vh[:,user_col_num], vh[:,col])
        sims[matrix.columns[col]] = similarity
    sims = sorted(sims.items(), key=lambda x: x[1], reverse=True)
    return sims

In [241]:
def get_users_courses(user_id):
    return train_data.loc[train_data['user_id'] == user_id]['course_id'].values

In [242]:
#user_id = 4
#courses = get_users_courses(user_id)
#print("{}: {}".format(user_id, courses))
#sims = get_sims_by_user_id(user_id)
#for sim_user_id, sim in sims:
#    sim_courses = get_users_courses(sim_user_id)
#    print("{}: {}".format(sim_user_id, sim_courses))

In [243]:
def get_users_recomend(user_id):
    if is_new_user(user_id):
        return best_couses
    courses = get_users_courses(user_id)
    recomend = []
    sims = get_sims_by_user_id(user_id)
    for sim_user_id, sim in sims:
        sim_courses = get_users_courses(sim_user_id)
        for course_id in sim_courses:
            if (course_id not in courses) and (course_id not in recomend):
                recomend.append(course_id)
                if len(recomend) == 3:
                    return recomend
    return recomend

In [244]:
get_users_recomend(4)

[94, 7, 154]

In [245]:
best_couses = list(train_data.groupby('course_id').user_id.nunique().sort_values(ascending=False).head(3).keys())
best_couses

[7, 1, 15]

In [167]:
is_new_user(3)

True

In [246]:
get_users_recomend(42)

[7, 1, 15]

In [247]:
df = pandas.read_csv('sample_submission.csv', sep=',')
df

,user_id,course_id_1,course_id_2,course_id_3
0,78,0,0,0
1,81,0,0,0
2,120,0,0,0
3,123,0,0,0
4,150,0,0,0
...,...,...,...,...
2895,185864,0,0,0
2896,186262,0,0,0
2897,186691,0,0,0
2898,186887,0,0,0


In [248]:
#df['recomend'] = get_users_recomend(df['user_id'])
df.insert(4, 'recomend', df['user_id'].apply(get_users_recomend))
df

,user_id,course_id_1,course_id_2,course_id_3,recomend
0,78,0,0,0,"[94, 7, 154]"
1,81,0,0,0,"[94, 7, 154]"
2,120,0,0,0,"[94, 7, 154]"
3,123,0,0,0,"[7, 1, 15]"
4,150,0,0,0,"[94, 7, 154]"
...,...,...,...,...,...
2895,185864,0,0,0,"[7, 1, 15]"
2896,186262,0,0,0,"[7, 1, 15]"
2897,186691,0,0,0,"[7, 1, 15]"
2898,186887,0,0,0,"[7, 1, 15]"


In [249]:
df['course_id_1'] = df.apply(lambda row: row['recomend'][0], axis = 1)
df['course_id_2'] = df.apply(lambda row: row['recomend'][1], axis = 1)
df['course_id_3'] = df.apply(lambda row: row['recomend'][2], axis = 1)
df

,user_id,course_id_1,course_id_2,course_id_3,recomend
0,78,94,7,154,"[94, 7, 154]"
1,81,94,7,154,"[94, 7, 154]"
2,120,94,7,154,"[94, 7, 154]"
3,123,7,1,15,"[7, 1, 15]"
4,150,94,7,154,"[94, 7, 154]"
...,...,...,...,...,...
2895,185864,7,1,15,"[7, 1, 15]"
2896,186262,7,1,15,"[7, 1, 15]"
2897,186691,7,1,15,"[7, 1, 15]"
2898,186887,7,1,15,"[7, 1, 15]"


In [250]:
df.to_csv('submission-F-03.csv', sep=',', index=False, columns=['user_id', 'course_id_1', 'course_id_2', 'course_id_3'])

In [196]:
#list(train_data.groupby('course_id').user_id.nunique().sort_values(ascending=False).head(30).keys())